In [2]:
!pip install tensorboard


Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.8 


In [30]:
%reload_ext tensorboard
%load_ext tensorboard
%tensorboard --logdir ../runs 

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [26]:
!kill 12127  # Kill the current TB process


In [31]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

# Load logs from a run directory
event_acc = EventAccumulator('./runs/exp1')
event_acc.Reload()  # Loads event data

# Get all tags
print(event_acc.Tags())

# Read scalar events
scalars = event_acc.Scalars('Loss/train')

# Example: print step and value
for s in scalars:
    print(f"Step: {s.step}, Value: {s.value}")

Trying to create a Glue session for the kernel.
Session Type: etl
Session ID: d6c9d7c3-8135-4172-a27d-c69363210ea0
Applying the following default arguments:
--glue_kernel_version 1.0.8
--enable-glue-datacatalog true


Following exception encountered while creating session: An error occurred (AccessDeniedException) when calling the CreateSession operation: User: arn:aws:sts::513997065974:assumed-role/AmazonSageMaker-ExecutionRole-20240506T092696/SageMaker is not authorized to perform: glue:CreateSession on resource: arn:aws:glue:eu-west-1:513997065974:session/d6c9d7c3-8135-4172-a27d-c69363210ea0 because no identity-based policy allows the glue:CreateSession action 

Error message: User: arn:aws:sts::513997065974:assumed-role/AmazonSageMaker-ExecutionRole-20240506T092696/SageMaker is not authorized to perform: glue:CreateSession on resource: arn:aws:glue:eu-west-1:513997065974:session/d6c9d7c3-8135-4172-a27d-c69363210ea0 because no identity-based policy allows the glue:CreateSession action 

Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/aws_glue_interactive_sessions_kernel/glue_kernel_utils/KernelGateway.py", line 104, in create_session
    response = self.glue_cli